In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/kaggle/input/resume-dataset-for-resume-ranking-group-10/resume_data.csv")

In [3]:
df.shape

(9544, 35)

# Removing Null Values

In [4]:
null_summary = pd.DataFrame({
    'Column Name': df.columns,
    'Null Count': df.isnull().sum().values
})

null_summary = null_summary[null_summary['Null Count'] > 0]

print(null_summary)

                            Column Name  Null Count
0                               address        8760
1                      career_objective        4804
2                                skills          56
3          educational_institution_name          84
4                          degree_names          84
5                         passing_years          84
6                   educational_results          84
7                          result_types          84
8                major_field_of_studies          84
9            professional_company_names          84
10                         company_urls          84
11                          start_dates          84
12                            end_dates          84
13                 related_skils_in_job          84
14                            positions          84
15                            locations          84
17      extra_curricular_activity_types        6118
18  extra_curricular_organization_names        6118
19  extra_cu

In [5]:
# List of the columns with 84 nulls
columns_with_84_nulls = [
    'educational_institution_name',
    'degree_names',
    'passing_years',
    'educational_results',
    'result_types',
    'major_field_of_studies',
    'professional_company_names',
    'company_urls',
    'start_dates',
    'end_dates',
    'related_skils_in_job',
    'positions',
    'locations'
]

common_null_rows = df[columns_with_84_nulls].isnull().all(axis=1)

count_common_null_rows = common_null_rows.sum()

print(f"Number of rows where ALL these columns are null: {count_common_null_rows}")


Number of rows where ALL these columns are null: 0


In [6]:
df[common_null_rows]

,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,online_links,issue_dates,expiry_dates,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score


In [7]:
# cols = [
#     'degree_names',
#     'major_field_of_studies',
#     'start_dates',
#     'end_dates',
# ]

# df = df.dropna(subset=cols)

In [8]:
df.shape

(9544, 35)

In [9]:
# null_summary = pd.DataFrame({
#     'Column Name': df.columns,
#     'Null Count': df.isnull().sum().values
# })

# null_summary = null_summary[null_summary['Null Count'] > 0]

# print(null_summary)

# Major Field of Study

In [10]:
import pandas as pd
import re

def clean_major_fields(df, column_name='major_field_of_studies'):
    df_cleaned = df.copy()

    abbreviations = {
        "cse": "computer science",
        "cs": "computer science",
        "it": "information technology",
        "computer": "computer science",
        "computers": "computer science",
        "ai": "artificial intelligence",
        "ml": "machine learning",
        "ds": "data science",
        "ece": "electronics engineering",
        "eee": "electrical engineering",
        "ee": "electrical engineering",
        "electrical": "electrical engineering",
        "electronics": "electronics engineering",
        "me": "mechanical engineering",
        "ce": "civil engineering",
        "che": "chemical engineering",
        "mechanical": "mechanical engineering",
        "civil": "civil engineering",
        "chemical": "chemical engineering",
        "finance": "finance",
        "accounting": "accounting",
        "business": "business administration",
        "management": "business administration",
        "marketing": "marketing",
        "statistics": "statistics",
        "economics": "economics",
        "biology": "biology",
        "chemistry": "chemistry",
        "physics": "physics",
        "math": "maths",
        "mathematics": "maths"
    }

    split_pattern = re.compile(r"[\/,&;|\s]+")

    empty_values = {"n/a", "none", "na", "null", "", "nan", "n, a", "n,a", "n", "a"}

    def process_value(value):
        if pd.isna(value) or value is None:
            return []
        
        value_str = str(value).lower().strip()

        if value_str in empty_values:
            return []

        value_str = value_str.strip("[]\"'")

        items = [item.strip() for item in split_pattern.split(value_str) if item.strip()]

        cleaned_items = []
        for item in items:
            if item in empty_values:
                continue
            cleaned_item = abbreviations.get(item, item)
            if cleaned_item:
                cleaned_items.append(cleaned_item)

        seen = set()
        unique_items = [x for x in cleaned_items if not (x in seen or seen.add(x))]

        return unique_items if unique_items else []

    df_cleaned[column_name] = df_cleaned[column_name].apply(process_value)
    
    return df_cleaned

In [11]:
def clean_experience_min_only(df, column_name='experience', default_value=0):
    """
    Cleans experience column to extract the minimum number of years mentioned.
    Examples:
        'At least 3 years' → 3
        '3 to 5 years' → 3
        '1 to 2 years' → 1
        NaN or invalid → default_value (e.g., 0)

    Args:
        df (pd.DataFrame): Input DataFrame
        column_name (str): The column name to clean
        default_value (int): Value to assign to missing/invalid entries

    Returns:
        pd.DataFrame: DataFrame with cleaned experience column
    """
    def extract_min_years(value):
        if pd.isna(value):
            return default_value
        # Find all numbers in the string
        numbers = re.findall(r'\d+', str(value))
        if numbers:
            return int(numbers[0])  # take the minimum
        return default_value

    df_cleaned = df.copy()
    df_cleaned[column_name] = df_cleaned[column_name].apply(extract_min_years)
    return df_cleaned

In [12]:
def clean_degree_name(text):
    if pd.isna(text):
        return ''
    
    text = text.lower()

    # Abbreviation expansion
    replacements = {
    r'\bb[\s\.-]*tech\b': 'bachelor of technology',
    r'\bm[\s\.-]*tech\b': 'master of technology',
    r'\bb[\s\.-]*sc\b': 'bachelor of science',
    r'\bm[\s\.-]*sc\b': 'master of science',
    r'\bb[\s\.-]*e\b': 'bachelor of engineering',
    r'\bm[\s\.-]*e\b': 'master of engineering',
    r'\bb[\s\.-]*a\b': 'bachelor of arts',
    r'\bm[\s\.-]*a\b': 'master of arts',
    r'\bb[\s\.-]*com\b': 'bachelor of commerce',
    r'\bm[\s\.-]*com\b': 'master of commerce',
    r'\bb[\s\.-]*ba\b': 'bachelor of business administration',
    r'\bm[\s\.-]*ba\b': 'master of business administration',
    r'\bmba\b': 'master of business administration',
    r'\bbba\b': 'bachelor of business administration',
    r'\bph[\s\.-]*d\b': 'doctor of philosophy',
    r'\bd[\s\.-]*phil\b': 'doctor of philosophy',
    }


    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [13]:
def clean_educational_requirement(text):
    if pd.isna(text):
        return ''
    
    text = text.lower()

    # Use same abbreviation expansion
    replacements = {
        r'\bb[\s\.-]*sc\b': 'bachelor of science',
        r'\bm[\s\.-]*sc\b': 'master of science',
        r'\bb[\s\.-]*tech\b': 'bachelor of technology',
        r'\bm[\s\.-]*tech\b': 'master of technology',
        r'\bb[\s\.-]*e\b': 'bachelor of engineering',
        r'\bm[\s\.-]*e\b': 'master of engineering',
        r'\bb[\s\.-]*a\b': 'bachelor of arts',
        r'\bm[\s\.-]*a\b': 'master of arts',
        r'\bb[\s\.-]*com\b': 'bachelor of commerce',
        r'\bm[\s\.-]*com\b': 'master of commerce',
        r'\bb[\s\.-]*ba\b': 'bachelor of business administration',
        r'\bm[\s\.-]*ba\b': 'master of business administration',
        r'\bmba\b': 'master of business administration',
        r'\bbba\b': 'bachelor of business administration',
        r'\bph[\s\.-]*d\b': 'doctor of philosophy',
        r'\bd[\s\.-]*phil\b': 'doctor of philosophy',
        # Other useful substitutions
        r'\bbachelor/honors\b': 'bachelor degree',
        r'\bdiploma\b': 'diploma',
        r'\bmasters?\b': 'master degree',  # handles "Masters" or "Master"
    }

    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)

    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [14]:
def clean_list_column(entry):
    """
    Converts list or list-like string entries to a space-separated string.
    """
    import ast

    if isinstance(entry, list):
        return ' '.join(entry).lower()
    elif isinstance(entry, str):
        try:
            parsed = ast.literal_eval(entry)
            if isinstance(parsed, list):
                return ' '.join(parsed).lower()
        except:
            pass
        return entry.lower()
    return ''

def combine_degree_and_major(degree, major):
    """
    Combines cleaned degree and major fields into one string for similarity comparison.
    """
    degree_str = clean_list_column(degree)
    major_str = clean_list_column(major)

    if degree_str and major_str:
        return f"{degree_str} in {major_str}"
    elif degree_str:
        return degree_str
    elif major_str:
        return major_str
    else:
        return ''

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Example: Use your actual DataFrame here
# df['combined_education'] already prepared
# df['educational_requirement_cleaned'] also cleaned (lowercase, no punctuation, etc.)

def compute_cosine_similarity(row, vectorizer):
    texts = [row['combined_education'], row['educational_requirements_cleaned']]
    tfidf = vectorizer.fit_transform(texts)
    similarity = cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]
    return similarity

In [16]:
df = clean_major_fields(df, column_name='major_field_of_studies')
df = clean_experience_min_only(df, column_name='experiencere_requirement')
df['degree_names_cleaned'] = df['degree_names'].apply(clean_degree_name)
df['educational_requirements_cleaned'] = df['educationaL_requirements'].apply(clean_educational_requirement)
df['combined_education'] = df.apply(
    lambda row: combine_degree_and_major(row['degree_names_cleaned'], row['major_field_of_studies']),axis=1)

In [17]:
# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Compute cosine similarity for each row
df['education_similarity'] = df.apply(
    lambda row: compute_cosine_similarity(row, tfidf_vectorizer), axis=1
)

# Cleaning experience_requirements

In [18]:
def clean_experience_min_only(df, column_name='experience', default_value=0):
    """
    Cleans experience column to extract the minimum number of years mentioned.
    Examples:
        'At least 3 years' → 3
        '3 to 5 years' → 3
        '1 to 2 years' → 1
        NaN or invalid → default_value (e.g., 0)

    Args:
        df (pd.DataFrame): Input DataFrame
        column_name (str): The column name to clean
        default_value (int): Value to assign to missing/invalid entries

    Returns:
        pd.DataFrame: DataFrame with cleaned experience column
    """
    def extract_min_years(value):
        if pd.isna(value):
            return default_value
        # Find all numbers in the string
        numbers = re.findall(r'\d+', str(value))
        if numbers:
            return int(numbers[0])  # take the minimum
        return default_value

    df_cleaned = df.copy()
    df_cleaned[column_name] = df_cleaned[column_name].apply(extract_min_years)
    return df_cleaned

# Job Experience

In [19]:
import ast

def remove_na_and_none_from_list(col):
    def clean(entry):
        try:
            # if already a list, skip parsing
            if isinstance(entry, list):
                items = entry
            else:
                items = ast.literal_eval(entry)
            return [i for i in items if str(i).strip().upper() not in ['N/A', 'NONE'] and i is not None]
        except:
            return entry  # return original if parsing fails
    return col.apply(clean)

df['start_dates'] = remove_na_and_none_from_list(df['start_dates'])
df['end_dates'] = remove_na_and_none_from_list(df['end_dates'])

In [20]:
import re

def replace_seasons_with_months(col):
    # Mapping of season to month
    season_to_month = {
        'spring': '03',
        'summer': '06',
        'fall': '09',
        'autumn': '09',
        'winter': '12',
    }

    def replace_season(entry):
        try:
            items = ast.literal_eval(entry) if isinstance(entry, str) else entry
            updated_items = []
            for item in items:
                if item is None:
                    updated_items.append(item)
                    continue
                # Match season followed by year, e.g., "Summer 2013"
                match = re.match(r'(?i)\b(spring|summer|fall|autumn|winter)\b\s+(\d{4})', str(item).strip())
                if match:
                    season = match.group(1).lower()
                    year = match.group(2)
                    month = season_to_month.get(season)
                    updated_items.append(f"{month}/{year}")
                else:
                    updated_items.append(item)
            return updated_items
        except:
            return entry  # keep original if parsing fails

    return col.apply(replace_season)

df['start_dates'] = replace_seasons_with_months(df['start_dates'])
df['end_dates'] = replace_seasons_with_months(df['end_dates'])

In [21]:
from datetime import datetime
import ast

def replace_current_terms_with_today(col):
    today_str = datetime.today().strftime('%b %d %Y')  # Example: 'Apr 22 2025'
    keywords = {'till date', 'current', 'ongoing', 'present', '∞'}

    def clean(entry):
        try:
            items = ast.literal_eval(entry) if isinstance(entry, str) else entry
            return [
                today_str if isinstance(i, str) and i.strip().lower() in keywords else i
                for i in items
            ]
        except:
            return entry  # return as-is if parsing fails

    return col.apply(clean)

# Apply it
df['start_dates'] = replace_current_terms_with_today(df['start_dates'])
df['end_dates'] = replace_current_terms_with_today(df['end_dates'])

filter and display rows where either start_dates or end_dates columns contains any item with the substring"20XX"

In [22]:
import ast

def contains_20xx(entry):
    try:
        items = ast.literal_eval(entry) if isinstance(entry, str) else entry
        return any(isinstance(i, str) and '20xx' in i.lower() for i in items)
    except:
        return False

mask = df['start_dates'].apply(contains_20xx) | df['end_dates'].apply(contains_20xx)
df_with_20xx = df.loc[mask, ['start_dates', 'end_dates']]

print(df_with_20xx)

                 start_dates                         end_dates
81    [May 20XX, April 20XX]           [August 20XX, May 20XX]
180       [20xx, 20xx, 20xx]  [May 06 2025, May 06 2025, 20xx]
222   [May 20XX, April 20XX]           [August 20XX, May 20XX]
548       [20xx, 20xx, 20xx]  [May 06 2025, May 06 2025, 20xx]
632         [September 20XX]                      [April 20XX]
...                      ...                               ...
9267        [September 20XX]                      [April 20XX]
9324        [September 20XX]                      [April 20XX]
9373  [May 20XX, April 20XX]           [August 20XX, May 20XX]
9453        [September 20XX]                      [April 20XX]
9508      [20xx, 20xx, 20xx]  [May 06 2025, May 06 2025, 20xx]

[84 rows x 2 columns]


In [23]:
import ast

def replace_array_with_unknown(entry):
    try:
        items = ast.literal_eval(entry) if isinstance(entry, str) else entry
        if any(isinstance(i, str) and '20xx' in i.lower() for i in items):
            return 'unknown'  # Replace entire array with the string 'unknown'
        return items
    except:
        return entry

df['start_dates'] = df['start_dates'].apply(replace_array_with_unknown)
df['end_dates'] = df['end_dates'].apply(replace_array_with_unknown)

replace each record in start_dates and end_dates with 'unknown' if either one of them contains an empty array.

In [24]:
import ast

def replace_empty_array_with_unknown(row):
    try:
        # Parse the entries if they are string representations of lists
        start = ast.literal_eval(row['start_dates']) if isinstance(row['start_dates'], str) else row['start_dates']
        end = ast.literal_eval(row['end_dates']) if isinstance(row['end_dates'], str) else row['end_dates']
        
        # Check if either start or end is an empty list
        if isinstance(start, list) and len(start) == 0 or isinstance(end, list) and len(end) == 0:
            return pd.Series({'start_dates': 'unknown', 'end_dates': 'unknown'})
        else:
            return pd.Series({'start_dates': row['start_dates'], 'end_dates': row['end_dates']})
    except:
        # In case of any parsing error, return the original values
        return pd.Series({'start_dates': row['start_dates'], 'end_dates': row['end_dates']})

# Apply the function to each row
df[['start_dates', 'end_dates']] = df.apply(replace_empty_array_with_unknown, axis=1)

converting all dates in string format to YYYY-MM

In [25]:
import numpy as np
import pandas as pd
from dateutil import parser
import ast

def standardize_date_array(entry):
    if isinstance(entry, str) and entry == 'unknown':
        return entry
    if isinstance(entry, float) and pd.isna(entry):
        return entry

    try:
        items = ast.literal_eval(entry) if isinstance(entry, str) else entry
        if not isinstance(items, list):  # guard clause
            return entry

        cleaned = []
        for item in items:
            if isinstance(item, str):
                try:
                    parsed = parser.parse(item, fuzzy=True)
                    cleaned.append(parsed.strftime('%Y-%m'))
                except:
                    cleaned.append(item)
            else:
                cleaned.append(item)
        return cleaned
    except:
        return entry
    
df['start_dates'] = df['start_dates'].apply(standardize_date_array)
df['end_dates'] = df['end_dates'].apply(standardize_date_array)

check if there are arrays with unequal length

In [26]:
import ast

def count_unequal_length_date_arrays(df):
    count = 0
    for start, end in zip(df['start_dates'], df['end_dates']):
        if isinstance(start, list) and isinstance(end, list):
            if len(start) != len(end):
                count += 1
    return count

unequal_length_count = count_unequal_length_date_arrays(df)
print("Total records with unequal-length date arrays:", unequal_length_count)

Total records with unequal-length date arrays: 452


replace the entries having unequal array length with string 'unknown'.

In [27]:
def replace_unequal_length_records(df):
    for idx, (start, end) in enumerate(zip(df['start_dates'], df['end_dates'])):
        if isinstance(start, list) and isinstance(end, list):
            if len(start) != len(end):
                df.at[idx, 'start_dates'] = 'unknown'
                df.at[idx, 'end_dates'] = 'unknown'
    return df

# Apply the function to your dataframe
df = replace_unequal_length_records(df)

sorting the dates arrays in accending order in both columns

In [28]:
from datetime import datetime

def sort_dates_in_array(entry):
    try:
        # Convert string dates to datetime objects for sorting
        items = ast.literal_eval(entry) if isinstance(entry, str) else entry
        # Sort the items based on datetime conversion
        sorted_items = sorted(items, key=lambda x: datetime.strptime(x, '%b %Y') if ' ' in x else datetime.strptime(x, '%m/%Y') if '/' in x else datetime.strptime(x, '%Y-%m'))
        return sorted_items
    except:
        return entry  # If any error occurs, return the original entry

# Apply the sorting function to both start_dates and end_dates
df['start_dates'] = df['start_dates'].apply(sort_dates_in_array)
df['end_dates'] = df['end_dates'].apply(sort_dates_in_array)
from datetime import datetime

def sort_dates_in_array(entry):
    try:
        # Convert string dates to datetime objects for sorting
        items = ast.literal_eval(entry) if isinstance(entry, str) else entry
        # Sort the items based on datetime conversion
        sorted_items = sorted(items, key=lambda x: datetime.strptime(x, '%b %Y') if ' ' in x else datetime.strptime(x, '%m/%Y') if '/' in x else datetime.strptime(x, '%Y-%m'))
        return sorted_items
    except:
        return entry  # If any error occurs, return the original entry

# Apply the sorting function to both start_dates and end_dates
df['start_dates'] = df['start_dates'].apply(sort_dates_in_array)
df['end_dates'] = df['end_dates'].apply(sort_dates_in_array)

In [29]:
def calculate_experience(start_dates, end_dates):
    # Check for 'unknown' in strings
    if isinstance(start_dates, str) and start_dates.lower() == 'unknown':
        return 'unknown'
    if isinstance(end_dates, str) and end_dates.lower() == 'unknown':
        return 'unknown'

    # Check for 'unknown' inside lists
    if isinstance(start_dates, list) and any(str(d).lower() == 'unknown' for d in start_dates):
        return 'unknown'
    if isinstance(end_dates, list) and any(str(d).lower() == 'unknown' for d in end_dates):
        return 'unknown'

    # Check for NaNs safely
    if isinstance(start_dates, list):
        if any(pd.isna(d) for d in start_dates):
            return np.nan
    elif pd.isna(start_dates):
        return np.nan

    if isinstance(end_dates, list):
        if any(pd.isna(d) for d in end_dates):
            return np.nan
    elif pd.isna(end_dates):
        return np.nan

    # Calculate total years of experience
    total_years = 0
    for start, end in zip(start_dates, end_dates):
        try:
            start_dt = datetime.strptime(str(start).strip(), '%Y-%m')
            end_dt = datetime.strptime(str(end).strip(), '%Y-%m')
            total_years += (end_dt - start_dt).days / 365.25
        except Exception:
            return np.nan  # Invalid date format

    return round(total_years, 2) if total_years != 0 else np.nan

df['experience_years'] = df.apply(
    lambda row: calculate_experience(row['start_dates'], row['end_dates']),
    axis=1
)

In [30]:
# Count how many 'unknown' strings are in the experience_years column
unknown_experience_count = (df['experience_years']=='unknown').sum()

print("Total 'unknown' in experience_years:", unknown_experience_count)

Total 'unknown' in experience_years: 788


total 'unknown' s are 788 and total 'nan's 112.

In [31]:
df['experience_years'].unique()

array([5.5, 5.66, 6.92, 13.83, 17.33, 18.34, 0.67, 19.33, 5.92, 6.33,
       33.33, 5.08, 5.25, 0.17, 4.33, 4.91, 'unknown', 5.42, 6.17, 22.75,
       27.0, 7.0, 13.0, 30.49, 1.0, 2.58, 33.25, 6.0, 17.5, 6.41, 0.75,
       1.42, 26.83, 16.08, 15.0, 3.09, 2.32, 13.16, 1.67, 0.84, 14.33,
       40.08, 8.5, 3.5, 13.67, 6.08, 7.33, 25.5, 0.5, 22.0, 24.83, 4.75,
       nan, 25.66, 19.17, 0.25, 18.25, 1.17, 3.67, 0.42, 3.58, 22.16,
       15.25, 18.92, 41.33, 35.67, 34.33, 33.0, 26.17, 62.0, 1.5, 19.83,
       5.33, 23.5, 10.0, 12.24, 33.5, 4.01, 9.67, 37.92, 14.59, 26.75,
       1.75, 0.33, 0.59, 23.58, 14.41, 33.83, 30.75, 11.92, 1.24, 38.66,
       6.25, 18.58, 5.83, 4.59, 10.92, 14.5, 19.66, 51.07, 18.66, 12.5,
       2.42, 13.91, 40.0, 6.16, 16.16, 0.76, 6.75, 8.08, 15.75, 35.24,
       19.5, 30.24, 10.5, 51.0, 17.75, 27.99, 20.33, 1.25, 0.58, 28.08,
       19.42, 2.83, 33.08, 8.25, 23.24, 0.92, 12.83, 2.59, 3.17, 18.83,
       26.92, 3.75, 19.08, 5.58, 21.58, 3.0, 4.0, 8.84, 24.66, 20.

In [32]:
df = df[df['experience_years'] != 'unknown'] #removing 'unkown'set
df = df.dropna(subset=['experience_years']) # removing null value records

In [33]:
df['experience_years'] = pd.to_numeric(df['experience_years'], errors='coerce')

In [34]:
df['experience_years'].describe()

count    8644.000000
mean       11.934044
std        11.659750
min         0.080000
25%         3.500000
50%         6.250000
75%        19.330000
max        62.000000
Name: experience_years, dtype: float64

# Cleaning Skills

In [35]:
import ast
import re

def clean_and_merge_skills(df, col1, col2, col3, new_col='merged_skills'):
    # Helper: Parse and clean a single cell
    def parse_and_clean(cell):
        try:
            parsed = ast.literal_eval(cell)
        except Exception:
            return []

        def flatten(x):
            if isinstance(x, list):
                return [item for sub in x for item in flatten(sub)]
            return [x]

        flat = flatten(parsed)
        cleaned = [s.strip().lower() for s in flat if isinstance(s, str)]
        return cleaned

    # Helper: Check if the list only contains generic skill patterns
    def is_generic_skills_list(skill_list):
        pattern = re.compile(r'^skill\s*\d+$')
        return all(pattern.match(s) for s in skill_list) and len(skill_list) > 0

    # Clean both columns
    cleaned_col1 = df[col1].apply(parse_and_clean)
    cleaned_col2 = df[col2].apply(parse_and_clean)
    cleaned_col3 = df[col3].apply(parse_and_clean)

    # Merge and deduplicate
    df[new_col] = [
        list(dict.fromkeys(c1 + c2 + c3)) for c1, c2, c3 in zip(cleaned_col1, cleaned_col2, cleaned_col3)
    ]

    # Remove generic skill lists
    df[new_col] = df[new_col].apply(lambda skills: [] if is_generic_skills_list(skills) else skills)

    # Drop original columns
    df.drop(columns=[col1, col2, col3], inplace=True)

    return df

clean_and_merge_skills(df, 'skills', 'related_skils_in_job', 'certification_skills', new_col='merged_skills')

,address,career_objective,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,company_urls,...,age_requirement,responsibilities.1,skills_required,matched_score,degree_names_cleaned,educational_requirements_cleaned,combined_education,education_similarity,experience_years,merged_skills
0,NaN,Big data analytics working and database wareho...,['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],[electronics engineering],['Coca-COla'],[None],...,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.850000,['bachelor of technology'],bachelor of science in computer science & engi...,bachelor of technology in electronics engineering,0.318784,5.50,"[big data, hadoop, hive, python, mapreduce, sp..."
1,NaN,Fresher looking to join as a data analyst and ...,"['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['2015', '2018']","['N/A', 'N/A']","['N/A', 'N/A']","[mathematics', 'statistics]",['BIB Consultancy'],['N/A'],...,NaN,Machine Learning Leadership\nCross-Functional ...,NaN,0.750000,"['bachelor of science (maths)', 'master of sci...",master degree of science in computer science &...,bachelor of science (maths) master of science ...,0.375598,5.66,"[data analysis, data analytics, business analy..."
2,NaN,NaN,"['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['2018'],['N/A'],['N/A'],"[electronics engineering, telecommunication]",['Axis Bank Limited'],['N/A'],...,NaN,"Trade Marketing Executive\nBrand Visibility, S...",Brand Promotion\nCampaign Management\nField Su...,0.416667,['bachelor of technology'],master degree of business administration (mast...,bachelor of technology in electronics engineer...,0.093495,6.92,"[software development, machine learning, deep ..."
3,NaN,To obtain a position in a fast-paced business ...,"['Martinez Adult Education, Business Training ...",['Computer Applications Specialist Certificate...,['2008'],[None],[None],"[computer science, applications]","['Company Name ï¼ City , State', 'Company Name...","[None, None, None, None, None, None]",...,Age 22 to 30 years,Apparel Sourcing\nQuality Garment Sourcing\nRe...,Fast typing skill\nIELTSInternet browsing & on...,0.760000,['computer applications specialist certificate...,bachelor degree,computer applications specialist certificate p...,0.000000,13.83,"[accounts payables, accounts receivables, acco..."
4,NaN,Professional accountant with an outstanding wo...,['Kent State University'],['Bachelor of Business Administration'],[None],['3.84'],[None],[accounting],"['Company Name', 'Company Name', 'Company Name...","[None, None, None, None, None]",...,NaN,iOS Lifecycle\nRequirement Analysis\nNative Fr...,iOS\niOS App Developer\niOS Application Develo...,0.650000,['bachelor of business administration'],bachelor of science (bachelor of science) in c...,bachelor of business administration in accounting,0.312103,17.33,"[analytical reasoning, compliance testing know..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9539,NaN,NaN,['Sanghvi College of Engineering'],['B.Tech'],['2019'],['N/A'],['N/A'],[],['BPM Foundation'],['N/A'],...,NaN,Data Platform Design\nData Pipeline Developmen...,Azure\nBig Data\nData Analytics\nETL Tools\nPo...,0.683333,['bachelor of technology'],bachelor of science (bachelor of science),bachelor of technology,0.503103,4.91,"[mathematical modelling, machine learning, pre..."
9540,NaN,Expertise EDA modeler. I like to learn what my...,"['KVoCT, Pune', 'KVoCT, Pune']","['B.CA', 'M.CA']","['2018', '2020']","[None, None]","[None, None]",[],['Passionate Solution'],[None],...,Age at most 40 years,Hardware & Software Installation\nSystem Monit...,NaN,0.650000,"['b.ca', 'm.ca']",bachelor of science (bachelor of science) in c...,b.ca m.ca,0.000000,5.25,"[data analysis, business analysis, machine lea..."
9541,NaN,Looking for roles related to ap

# Skills Required

In [36]:
def preprocess_skills_required_column(df):
    
    def process_skill_entry(entry):
        if pd.isna(entry) or entry == "":
            return []
        
        # Split by newlines and filter out empty strings
        skills = [s.strip() for s in entry.split('\n') if s.strip()]
        
        # Clean each skill
        cleaned_skills = []
        for skill in skills:
            # Remove bullet points and other unwanted characters
            skill = skill.replace('•', '').strip()
            if skill:  # Only add non-empty skills
                cleaned_skills.append(skill)
        
        # Remove duplicates while preserving order
        seen = set()
        unique_skills = [x for x in cleaned_skills if not (x in seen or seen.add(x))]
        
        return unique_skills
    
    # Apply the processing function to the skills_required column
    df['skills_required_list'] = df['skills_required'].apply(process_skill_entry)

    # Drop the original column
    df.drop(columns=['skills_required'], inplace=True)
    
    return df

df = preprocess_skills_required_column(df)

# Get the cosine similarity between Skills and Skills Required

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def compute_skill_cosine_similarity(df):
    # 1. Convert lists of skills to space-separated strings
    df['merged_skills_str'] = df['merged_skills'].apply(lambda lst: " ".join(lst))
    df['skills_required_str'] = df['skills_required_list'].apply(lambda lst: " ".join(lst))

    # 2. Prepare and fit a TF-IDF vectorizer on all skill-texts
    vectorizer = TfidfVectorizer()
    all_skill_texts = df['merged_skills_str'].tolist() + df['skills_required_str'].tolist()
    vectorizer.fit(all_skill_texts)

    # 3. Transform each column of skill-strings into TF-IDF vectors
    tfidf_merged = vectorizer.transform(df['merged_skills_str'])
    tfidf_required = vectorizer.transform(df['skills_required_str'])

    # 4. Compute cosine similarity for each row
    similarity_scores = []
    for i in range(len(df)):
        vec1 = tfidf_merged[i]
        vec2 = tfidf_required[i]
        sim_score = cosine_similarity(vec1, vec2)[0][0]
        similarity_scores.append(sim_score)

    # Add the similarity scores as a new column in the DataFrame
    df['cosine_similarity_skills'] = similarity_scores

    # Drop intermediate columns
    df = df.drop(['merged_skills_str', 'skills_required_str'], axis=1)

    return df

df = compute_skill_cosine_similarity(df)

# Degree Names

In [38]:
import re

EDUCATION_ALIASES = {
    "phd": ["doctor of philosophy", "ph.d", "ph.d.", "phd", "doctorate", "ph.d. in", "phd candidate"],
    "mba": ["master of business administration", "mba executive", "executive mba", "mba", "masters of business administration"],
    "msc": ["master of science", "m.sc", "m.s", "masters of science", "msc", "masters in science", "m.sc."],
    "ma": ["master of arts", "m.a", "m.a.", "masters of arts"],
    "mcom": ["master of commerce", "m.com", "mcom"],
    "me": ["master of engineering", "m.e", "m.eng", "m.e.", "m.engg"],
    "mtech": ["master of technology", "m.tech", "mtech", "mtech integrated"],
    "bsc": ["bachelor of science", "b.sc", "b.s", "bsc", "b.sc.", "b.s.", "honours bachelor of science", "bachelors of science"],
    "ba": ["bachelor of arts", "b.a", "ba", "b.a.", "bachelors of arts"],
    "bcom": ["bachelor of commerce", "b.com", "bcom"],
    "be": ["bachelor of engineering", "b.e", "b.e.", "b.eng", "b.engg", "bachelor of engineering (b.e"],
    "btech": ["bachelor of technology", "b.tech", "b.tech.", "btech", "b.tech(computers)", "dual degree (b.tech + m.tech)", "integrated b.tech & m.tech"],
    "bba": ["bachelor of business administration", "b.b.a", "bba", "bba - accounting", "bba - finance", "bachelor business administration"],
    "bca": ["bachelor of computer applications", "b.c.a", "bca"],
    "mca": ["master of computer applications", "m.c.a", "mca"],
    "bs": ["bs", "b.s", "b.s.", "b.s in", "bachelor's degree in science", "bachelor's in science"],
    "ms": ["ms", "m.s", "m.s.", "master in computer science", "masters of science in information technology"],
    "aa": ["associate of arts", "a.a", "aa"],
    "aas": ["associate of applied science", "a.a.s", "aas"],
    "as": ["associate of science", "a.s", "as", "associate of science degree"],
    "associate": ["associate's degree", "associate degree", "associates degree", "associates", "associate"],
    "diploma": ["technical diploma", "associate diploma", "polytechnic diploma", "diploma", "general diploma", "pg diploma", "master's diploma"],
    "high school": ["high school diploma", "ged", "grade 12", "xii", "x", "kcse"],
    "certificate": ["certificate of completion", "graduate certificate", "business certification", "epa certification", "aws brazing certification", "skills", "course", "certification", "minor", "training", "coaching"],
    "others": ["n/a", "select one", "attending", "testing computer software", "general courses"],

    # Education levels that are more common in the Sri Lankan context
    "al": ["advanced level", "a/l", "a.l", "gce a/l", "gce advanced level", "gce (a/l)", "gce(al)", "gce-a/l"],
    "ol": ["ordinary level", "o/l", "o.l", "gce o/l", "gce ordinary level", "gce (o/l)", "gce(ol)", "gce-o/l"],
    "nvq": ["nvq", "nvq level 3", "nvq level 4", "nvq level 5", "nvq level 6", "national vocational qualification", "nvq diploma"],
    "hnd": ["hnd", "higher national diploma", "hnd in", "higher national diploma in"],
    "cima": ["cima", "chartered institute of management accountants", "cima qualification"],
    "acca": ["acca", "association of chartered certified accountants"],
    "ca": ["chartered accountant", "institute of chartered accountants of sri lanka", "ica", "ca sri lanka"],
    "slim": ["slim", "slim diploma", "sri lanka institute of marketing", "slim pgd"],
    "nibt": ["nibt", "national institute of business & technology", "nibt diploma"],
    "bit": ["bit", "bachelor of information technology", "bit degree", "bit (colombo university)"]

}

EDUCATION_RANKS = {
    "others": 0,
    
    "high school": 1,
    "certificate": 1,
    "ol": 1,
    
    "al": 2,
    
    "diploma": 3,
    "associate": 3,
    "nvq": 3,
    "hnd": 3,
    "aa": 3,
    "aas": 3,
    "as": 3,
    "slim": 3,  
    "nibt": 3,   
    
    "bsc": 4,
    "bs": 4,
    "ba": 4,
    "be": 4,
    "btech": 4,
    "bit": 4,    
    "cima": 4,   
    "acca": 4,      
    "bcom": 4,
    "bba": 4,
    "bca": 4,
    
    "msc": 5,
    "ms": 5,
    "ma": 5,
    "me": 5,
    "mtech": 5,
    "mcom": 5,
    "mba": 5,
    "mca": 5,
    "ca": 5,
    
    "phd": 6
}

# Flattenning the aliases for easy reverse lookup
FLATTENED_ALIASES = {}
for canonical, synonyms in EDUCATION_ALIASES.items():
    for synonym in synonyms:
        FLATTENED_ALIASES[synonym.lower()] = canonical


def get_highest_education(degree_entries):
    if not isinstance(degree_entries, list):
        degree_entries = [degree_entries]

    best_match = ("unknown", -1)

    for entry in degree_entries:
        if not isinstance(entry, str):
            continue

        # Clean and normalize text
        text = re.sub(r'[^\w\s]', '', entry.lower())

        # Try matching from aliases
        for synonym, canonical in FLATTENED_ALIASES.items():
            if synonym in text:
                rank = EDUCATION_RANKS.get(canonical, -1)
                if rank > best_match[1]:
                    best_match = (canonical, rank)

        # Fallback to direct canonical keyword matching
        for canonical, rank in EDUCATION_RANKS.items():
            if canonical in text:
                if rank > best_match[1]:
                    best_match = (canonical, rank)

    return best_match[1]

# Educational Requirements

In [39]:
import re

EDUCATION_ALIASES = {
    "phd": ["doctor of philosophy", "ph.d", "ph.d.", "phd", "doctorate", "ph.d. in", "phd candidate"],
    "mba": ["master of business administration", "mba executive", "executive mba", "mba", "masters of business administration"],
    "msc": ["master of science", "m.sc", "m.s", "masters of science", "msc", "masters in science", "m.sc."],
    "ma": ["master of arts", "m.a", "m.a.", "masters of arts"],
    "mcom": ["master of commerce", "m.com", "mcom"],
    "me": ["master of engineering", "m.e", "m.eng", "m.e.", "m.engg"],
    "mtech": ["master of technology", "m.tech", "mtech", "mtech integrated"],
    "bsc": ["bachelor of science", "b.sc", "b.s", "bsc", "b.sc.", "b.s.", "honours bachelor of science", "bachelors of science"],
    "ba": ["bachelor of arts", "b.a", "ba", "b.a.", "bachelors of arts"],
    "bcom": ["bachelor of commerce", "b.com", "bcom"],
    "be": ["bachelor of engineering", "b.e", "b.e.", "b.eng", "b.engg", "bachelor of engineering (b.e"],
    "btech": ["bachelor of technology", "b.tech", "b.tech.", "btech", "b.tech(computers)", "dual degree (b.tech + m.tech)", "integrated b.tech & m.tech"],
    "bba": ["bachelor of business administration", "b.b.a", "bba", "bba - accounting", "bba - finance", "bachelor business administration"],
    "bca": ["bachelor of computer applications", "b.c.a", "bca"],
    "mca": ["master of computer applications", "m.c.a", "mca"],
    "bs": ["bs", "b.s", "b.s.", "b.s in", "bachelor's degree in science", "bachelor's in science"],
    "ms": ["ms", "m.s", "m.s.", "master in computer science", "masters of science in information technology"],
    "aa": ["associate of arts", "a.a", "aa"],
    "aas": ["associate of applied science", "a.a.s", "aas"],
    "as": ["associate of science", "a.s", "as", "associate of science degree"],
    "associate": ["associate's degree", "associate degree", "associates degree", "associates", "associate"],
    "diploma": ["technical diploma", "associate diploma", "polytechnic diploma", "diploma", "general diploma", "pg diploma", "master's diploma"],
    "high school": ["high school diploma", "ged", "grade 12", "xii", "x", "kcse"],
    "certificate": ["certificate of completion", "graduate certificate", "business certification", "epa certification", "aws brazing certification", "skills", "course", "certification", "minor", "training", "coaching"],
    "others": ["n/a", "select one", "attending", "testing computer software", "general courses"],

    # Education levels that are more common in the Sri Lankan context
    "al": ["advanced level", "a/l", "a.l", "gce a/l", "gce advanced level", "gce (a/l)", "gce(al)", "gce-a/l"],
    "ol": ["ordinary level", "o/l", "o.l", "gce o/l", "gce ordinary level", "gce (o/l)", "gce(ol)", "gce-o/l"],
    "nvq": ["nvq", "nvq level 3", "nvq level 4", "nvq level 5", "nvq level 6", "national vocational qualification", "nvq diploma"],
    "hnd": ["hnd", "higher national diploma", "hnd in", "higher national diploma in"],
    "cima": ["cima", "chartered institute of management accountants", "cima qualification"],
    "acca": ["acca", "association of chartered certified accountants"],
    "ca": ["chartered accountant", "institute of chartered accountants of sri lanka", "ica", "ca sri lanka"],
    "slim": ["slim", "slim diploma", "sri lanka institute of marketing", "slim pgd"],
    "nibt": ["nibt", "national institute of business & technology", "nibt diploma"],
    "bit": ["bit", "bachelor of information technology", "bit degree", "bit (colombo university)"]

}

# EDUCATION_RANKS = {
EDUCATION_RANKS = {
    "others": 0,
    
    "high school": 1,
    "certificate": 1,
    "ol": 1,
    
    "al": 2,
    
    "diploma": 3,
    "associate": 3,
    "nvq": 3,
    "hnd": 3,
    "aa": 3,
    "aas": 3,
    "as": 3,
    "slim": 3,  
    "nibt": 3,   
    
    "bsc": 4,
    "bs": 4,
    "ba": 4,
    "be": 4,
    "btech": 4,
    "bit": 4,    
    "cima": 4,   
    "acca": 4,      
    "bcom": 4,
    "bba": 4,
    "bca": 4,
    
    "msc": 5,
    "ms": 5,
    "ma": 5,
    "me": 5,
    "mtech": 5,
    "mcom": 5,
    "mba": 5,
    "mca": 5,
    "ca": 5,
    
    "phd": 6
}

In [40]:
# Flattenning the aliases for easy reverse lookup
FLATTENED_EDU_ALIASES = {
    synonym: canonical
    for canonical, synonyms in EDUCATION_ALIASES.items()
    for synonym in synonyms
}

def encode_ed_req(text):
    if not isinstance(text, str) or not text.strip():
        return EDUCATION_RANKS["others"]

    best_rank = EDUCATION_RANKS["others"]

    parts = re.split(r'[,\n/•;]', text.lower())

    for part in parts:
        part_clean = re.sub(r'[^\w\s]', '', part.strip())  # remove punctuation

        # Match from aliases
        for synonym, canonical in FLATTENED_EDU_ALIASES.items():
            if synonym in part_clean:
                rank = EDUCATION_RANKS.get(canonical, 0)
                best_rank = max(best_rank, rank)

        # Fallback direct canonical match
        for canonical, rank in EDUCATION_RANKS.items():
            if canonical in part_clean:
                best_rank = max(best_rank, rank)

    return best_rank

# Cleaning the Training Dataset

In [41]:
# Clearing null value records in the following two columns
cols = [
    'degree_names',
    'major_field_of_studies',
]

df = df.dropna(subset=cols)

In [42]:
df['highest_degree'] = df['degree_names'].apply(get_highest_education)
df['ed_req_encoded'] = df['educationaL_requirements'].apply(encode_ed_req)

# df_cleaned = clean_major_fields(df, column_name='major_field_of_studies')
# df['major_field_encoded'] = df_cleaned['major_field_of_studies']

df_cleaned = clean_experience_min_only(df, column_name='experiencere_requirement')
df['exp_req_encoded'] = df['experiencere_requirement']

In [43]:
df = df.drop(['start_dates', 'end_dates'], axis=1)

In [44]:
df.columns

Index(['address', 'career_objective', 'educational_institution_name',
       'degree_names', 'passing_years', 'educational_results', 'result_types',
       'major_field_of_studies', 'professional_company_names', 'company_urls',
       'positions', 'locations', 'responsibilities',
       'extra_curricular_activity_types',
       'extra_curricular_organization_names',
       'extra_curricular_organization_links', 'role_positions', 'languages',
       'proficiency_levels', 'certification_providers', 'online_links',
       'issue_dates', 'expiry_dates', '﻿job_position_name',
       'educationaL_requirements', 'experiencere_requirement',
       'age_requirement', 'responsibilities.1', 'matched_score',
       'degree_names_cleaned', 'educational_requirements_cleaned',
       'combined_education', 'education_similarity', 'experience_years',
       'merged_skills', 'skills_required_list', 'cosine_similarity_skills',
       'highest_degree', 'ed_req_encoded', 'exp_req_encoded'],
      dtype='ob

In [45]:
df = df.drop([
    'address', 'career_objective', 'educational_institution_name', 'degree_names', 'passing_years', 
    'educational_results', 'result_types', 'major_field_of_studies', 'professional_company_names', 
    'company_urls','positions', 'locations', 'responsibilities', 'extra_curricular_activity_types', 
    'extra_curricular_organization_names', 'extra_curricular_organization_links', 'role_positions', 
    'languages', 'proficiency_levels', 'certification_providers', 'online_links', 'issue_dates', 
    'expiry_dates', '﻿job_position_name', 'educationaL_requirements', 'experiencere_requirement', 
    'age_requirement', 'responsibilities.1', 'merged_skills', 'skills_required_list',
    'degree_names_cleaned', 'educational_requirements_cleaned', 'combined_education'
], axis=1)

In [46]:
df.head()

,matched_score,education_similarity,experience_years,cosine_similarity_skills,highest_degree,ed_req_encoded,exp_req_encoded
0,0.850000,0.318784,5.50,0.0,4,4,1
1,0.750000,0.375598,5.66,0.0,5,5,5
2,0.416667,0.093495,6.92,0.0,4,5,3
3,0.760000,0.000000,13.83,0.0,5,4,1
4,0.650000,0.312103,17.33,0.0,4,4,4


In [47]:
df.describe()

,matched_score,education_similarity,experience_years,cosine_similarity_skills,highest_degree,ed_req_encoded,exp_req_encoded
count,8560.000000,8560.000000,8560.000000,8560.000000,8560.000000,8560.000000,8560.000000
mean,0.661067,0.248449,11.845930,0.016344,4.370561,4.428855,3.035631
std,0.166082,0.195666,11.663942,0.037167,0.723529,0.494941,2.910274
min,0.000000,0.000000,0.080000,0.000000,-1.000000,4.000000,0.000000
25%,0.583333,0.108173,3.500000,0.000000,4.000000,4.000000,1.000000
50%,0.683333,0.194314,6.250000,0.000000,4.000000,4.000000,3.000000
75%,0.793333,0.342125,19.170000,0.017013,5.000000,5.000000,5.000000
max,0.950000,1.000000,62.000000,0.415107,6.000000,5.000000,15.000000


In [48]:
df.to_csv('df_cleaned.csv', index=False)